In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

url = "https://thewaltdisneycompany.com/investor-relations/"

response = requests.get(url)
response.raise_for_status()  

soup = BeautifulSoup(response.text, 'html.parser')

pdf_links = soup.find_all('a', class_='pdf-file')

save_dir = "disney_reports"
os.makedirs(save_dir, exist_ok=True)

# Choose what to download: 'annual' or 'quarterly'
report_type = 'annual' 

for link in pdf_links:
    pdf_url = link.get('href')
    if pdf_url.endswith('.pdf'):
        link_text = link.get_text(strip=True).lower() 
        
        if (report_type == 'annual' and 'annual report' in link_text) or \
           (report_type == 'quarterly' and ('q1' in link_text or 'q2' in link_text or 'q3' in link_text or 'q4' in link_text or 'earnings report' in link_text)):
            pdf_response = requests.get(pdf_url)
            pdf_response.raise_for_status()
            
            file_name = pdf_url.split('/')[-1]
            save_path = os.path.join(save_dir, file_name)
            
            with open(save_path, 'wb') as f:
                f.write(pdf_response.content)
            print(f"Downloaded {file_name}")

print(f"All {report_type} reports have been downloaded.")


In [ ]:
pip install PyMuPDF tabula-py pandas


In [ ]:
import os
import pandas as pd
from tabula import convert_into

def extract_tables(pdf_path, output_dir):
    output_path = os.path.join(output_dir, os.path.basename(pdf_path).replace('.pdf', '.csv'))
    try:
        convert_into(pdf_path, output_path, output_format="csv", pages="all")
        print(f"Tables extracted from {os.path.basename(pdf_path)} and saved as {output_path}")
    except Exception as e:
        print(f"Failed to extract tables from {os.path.basename(pdf_path)}: {e}")

def process_pdfs(pdf_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(pdf_dir):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            extract_tables(pdf_path, output_dir)

pdf_directory = 'disney_reports'
output_directory = 'extracted_reports'
process_pdfs(pdf_directory, output_directory)
